In [ ]:
from statsmodels.tsa.seasonal import STL
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import acf
from scipy.signal import find_peaks, peak_prominences
import numpy as np

from TSB_AD.utils.slidingWindows import find_length_rank


In [ ]:
def load_series(files):
    results = {}
    for f in files:
        df = pd.read_csv("benchmark_exp/TSB-AD/TSB-AD-U/" + f + ".csv")
        results[f.split(".")[0]] = df
    return results

gah = ['025_UCR_Anomaly_CIMIS44AirTemperature1_2046_5391',
       '029_UCR_Anomaly_ECG2_7500_16000',
       '033_UCR_Anomaly_Italianpowerdemand_7482_29480',
       '036_UCR_Anomaly_insectEPG3_2499_7000',
       '051_UCR_Anomaly_DISTORTEDresperation4_48812_128430',
       '080_UCR_Anomaly_gait1_16250_38500',
       '085_UCR_Anomaly_Italianpowerdemand_29895_39240',
       '107_UCR_Anomaly_gait3_16250_59900',
       '154_UCR_Anomaly_resperation11_48750_110800',
       '208_UCR_Anomaly_DISTORTEDresperation1_50000_110260',
       '216_UCR_Anomaly_resperation10_48750_130700',
       '228_UCR_Anomaly_taichidbS0715Master_50000_884100',
       '504_IOPS_KPI-8723f0fb-eaef-32e6-b372-6034c9c04b80_5578_5678',
       '506_IOPS_KPI-6d1114ae-be04-3c46-b5aa-be1a003a57cd_8620_8720',
       '507_IOPS_KPI-301c70d8-1630-35ac-8f96-bc1b6f4359ea_1483_1583',
       '508_IOPS_KPI-e0747cad-8dc8-38a9-a9ab-855b61f5551d_500_341',
       '509_IOPS_KPI-0efb375b-b902-3661-ab23-9a0bb799f4e3_500_264',
       '515_IOPS_KPI-ab216663-dcc2-3a24-b1ee-2c3e550e06c9_500_77',
       '517_IOPS_KPI-8723f0fb-eaef-32e6-b372-6034c9c04b80_5638_5738',
       '530_WSD_150_4521_4599', '532_WSD_171_643_743', '533_WSD_168_4521_6454',
       '534_WSD_187_4559_12588', '538_WSD_153_500_276',
       '540_WSD_104_2411_2511', '544_WSD_191_4559_9714', '545_WSD_37_500_78',
       '548_WSD_57_4521_6828', '549_WSD_154_1129_1229', '554_WSD_184_500_53']

res = load_series(['006_NAB_data_CloudWatch_11_1007_1437', '242_Yahoo_A2synthetic_66_data_500_893', '784_OPPORTUNITY_S4-ADL5_1100_1200_col_id_17_1100_1200',  '025_UCR_Anomaly_CIMIS44AirTemperature1_2046_5391'])
#res = load_series(gah)

In [ ]:
s = res['006_NAB_data_CloudWatch_11_1007_1437']

In [ ]:
def find_length(data):
    a, b = np.quantile(data, [0.001, 0.999])
    data = np.clip(data, a, b)
    
    #data = data[:min(20000, len(data))]
    n_lags = 5000
    base = 3
    auto_corr = acf(data, nlags=n_lags, fft=True)[base:]
    try:
        peaks, _ = find_peaks(auto_corr)
        prominences = peak_prominences(auto_corr, peaks)[0]
        peaks = peaks + base
        prominent_peak_idx = np.argmax(prominences)
    except:
        return []
    if prominences[prominent_peak_idx] < 0.1:
        result = []
    else:
        highest_peak = np.argmax(auto_corr[peaks - base])
        ac_of_prominent = auto_corr[peaks[prominent_peak_idx] - base]
        # 99% significance level of autocorrelation
        if ac_of_prominent < 2.576 / np.sqrt(len(data)):
            return []
        result = [peaks[prominent_peak_idx]]
        if highest_peak != prominent_peak_idx and prominences[highest_peak] > 0.1:
            result.append(peaks[highest_peak])
    return result

In [ ]:
window_length = find_length(s['Data'])[0]
window_length

In [ ]:
find_length_rank(s['Data'])

In [ ]:
fit = STL(s['Data'], period=window_length).fit()

In [ ]:
plt.plot(fit.seasonal + fit.trend)
s['Data'].plot(alpha=0.5, figsize=(30, 5))

In [ ]:
1 - (fit.resid ** 2).mean() / s['Data'].var()

In [ ]:
#pd.plotting.autocorrelation_plot(s['Data'])
#plt.hlines([2/np.sqrt(len(s))], xmin=0, xmax=1400, colors='r')

In [ ]:
from sklearn.metrics import r2_score
r2_score(s['Data'], fit.seasonal + fit.trend)

In [ ]:
1 - (fit.resid ** 2).mean() / s['Data'].var()

In [ ]:
def iqr(s):
    return s.quantile([0.75]).iloc[0] - s.quantile([0.25]).iloc[0]
    

In [ ]:
iqr(fit.observed)

In [ ]:
iqr(fit.trend)

In [ ]:
iqr(fit.seasonal)

In [ ]:
iqr(fit.resid)

In [ ]:
fit.plot();

In [ ]:
from TSB_AD.utils.slidingWindows import find_length_rank
from utils import plot_detection
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

In [ ]:
score = np.abs(fit.resid)

In [ ]:
plot_detection(s.Data, s.Label, scores=score, window_length=window_length)
plt.title(f"{average_precision_score(s.Label, score):.2f}  ({window_length})")

In [ ]:
s = res['784_OPPORTUNITY_S4-ADL5_1100_1200_col_id_17_1100_1200']

In [ ]:
window_length = find_length(s['Data'])

In [ ]:
window_length

In [ ]:
bla = acf(s['Data'], nlags=5000, fft=True)

In [ ]:
plt.plot(bla)

In [ ]:
res = STL(s['Data'], period=18).fit()
res.plot();

In [ ]:
r2_score(s['Data'], res.seasonal + res.trend)

In [ ]:
score = np.abs(res.resid)
plot_detection(s.Data, s.Label, scores=score, window_length=window_length)
plt.title(f"{average_precision_score(s.Label, score):.2f}  ({window_length})")

In [ ]:
res = STL(s['Data'], period=1286).fit()
res.plot();

In [ ]:
r2_score(s['Data'], res.seasonal + res.trend)

In [ ]:
1 - (res.resid ** 2).mean() / s['Data'].var()

In [ ]:
score = np.abs(res.resid)
plot_detection(s.Data, s.Label, scores=score, window_length=window_length)
plt.title(f"{average_precision_score(s.Label, score):.2f}  ({window_length})")

In [ ]:
res = STL(s['Data'], period=1286, trend_deg=0).fit()
res.plot();

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
es = ExponentialSmoothing(s['Data'], seasonal_periods=18, seasonal='add', trend='add').fit()

In [ ]:
plt.figure(figsize=(20, 10))

s['Data'].plot()
es.fittedvalues.plot(alpha=0.4)

In [ ]:
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing

In [ ]:
es = ExponentialSmoothing(s['Data'], seasonal=18).fit()
plt.figure(figsize=(20, 10))

s['Data'].plot()
es.fittedvalues.plot(alpha=0.4)

In [ ]:
es = ExponentialSmoothing(s['Data'], seasonal=1286).fit()
plt.figure(figsize=(20, 10))

s['Data'].plot()
es.fittedvalues.plot(alpha=0.4)